In [ ]:
# Basic Setup
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

1. Crawl Github with a specific search category to create our dataset

2. Take the code and remove functions and ask Copilot to finish the code

3. Label original code as human generated, label Copilot code as AI generated

4. Separate train, test, validation sets as 75%, 15%, 10% respectively

5. Tokenize all code (Word2Vec) and create features for our ML model to use

6. Run ML models (ex: Random Forest (RF), XGBoost, and Support Vector Machine (SVM) )

7. Gather precision, recall, accuracy, and f1-score for each model